In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [ ]:
# Custom functions to use

# Function to wrap angle in order to have values between 0 and 2pi
def wrapAngle(ang):
    x=np.mod(ang,(2*np.pi))
    return x

# Calculate the separations between an angular configuration of charges
def separation1(thetas):
    to_ret=[]
    for i in np.arange(1,len(thetas)):
        diff=np.abs(thetas[i]-thetas[i-1])
        if(diff>np.pi):
            to_ret.append(np.pi-diff%np.pi)
        else:
            to_ret.append(diff)
        
    return to_ret


In [ ]:
# State initial parameters for the simulation

# Initial number of particles 
N=100

# Number of iterations per simulation
N_iter=1500

# Set virtual time step
dt=0.5/N

# Set constant friction (f) coefficient
f=1

# Set temperatures (\beta) to simulate
betas=[1,2,4]

# Set fusion lengths (l_f) to simulate
lfs=[0.01,0.1]
#lfs=[0.001,0.01,0.1,0.5]
#lfs=[0.5,1,2,3,4,5,6,7,8,9,10,11,12]

# Number of simulations per specific parameters to save
N_sim=1000


# Start iteration over lfs and betas
for lf in lfs:
    
    # Calculate the mean separation depending on the initial number of particles
    sprom=2*np.pi/N
    
    # Set the critical fusion angle (\theta_f)
    fusion_tol=lf*0.1*sprom

    for beta in betas:
        
        # Path to save this precise parameters
        # If this path already exists, the program will not run (Have to work on this) 
        path1='N'+str(N)+'B'+str(beta)+'S'+str(lf)
        
        # Create first folder to save N_sim simulations
        os.makedirs(path1)

        sim=1

        while sim<=N_sim:
            # Reset time, fusion time and number of particles vector 
            t=0
            t_file=[]
            Nt=[]

            # Create vectors to evolve the simulation
            theta_now=[]
            theta_later=[]

            # Path to save data for the n-th simulation
            path2='sim'+str(sim)
            patht=path1+'/'+path2
            
            # Create folder to save the specific simulation sim
            os.makedirs(patht)

            # Initialize the simulation with the first configuration
            # Create random noise to add to the first configuration
            ruido=np.random.uniform(-0.5*np.pi/N,0.5*np.pi/N,N)
            # Initialize charges uniformly around the circle and add noise 
            first=np.linspace(0.0,2*np.pi-2*np.pi/N,N)+ruido
            first=np.sort(first)
            theta_now=first

            # Create vector and parameters for fusion events
            fusion_time=[]

            k=0
            while k<N_iter:
                theta_later=[]
                for i in np.arange(len(theta_now)):
                    sum_i=0

                    # Sum over all charges to get electric force (E(\theta))
                    for j in np.arange(len(theta_now)):
                        if(i!=j):
                            sum_i=sum_i+1/np.tan(0.5*(theta_now[i]-theta_now[j]))

                    # Calculate the mean and variance 
                    mu_i=0.5*sum_i*dt
                    sigma_i=np.sqrt(2*dt/(f*beta))
                    
                    # Generate the random step base on the latter values to evolve the system
                    step=np.random.normal(mu_i,sigma_i)

                    # Evolve the system and wrap the angles 
                    theta_to_append=wrapAngle(theta_now[i]+step)
                    
                    # Evolve the system
                    theta_later.append(theta_to_append)

                # Calculate the separation between charges with separation1
                ss=separation1(theta_later)

                # Let the system evolve 50 iterations
                # At the 50-th iteration start the fusion scheme with a random pair of consecutive charges
                if(k==50):
                    to_del=np.around(np.random.uniform(0,N,1))
                    theta_later=np.delete(theta_later,(to_del,to_del-1))
                
                # After the 50-th iteration check every step for fusion events
                for i in np.arange(len(ss)):
                    if(ss[i]<fusion_tol and k>50):
                        theta_later=np.delete(theta_later,(i,i-1))
                        
                        #Save the time for the fusion event
                        fusion_time.append(t)
                        break

                # Evolve the system
                theta_now=theta_later
                
                # Save the pertinent parameters of the time step
                Nt.append(len(theta_later))
                t_file.append(t)
                
                # Evolve the simulations
                t=t+dt
                k=k+1
            
            # Save the data on a file
            np.savetxt(patht+'/Nt.txt',Nt)    
            np.savetxt(patht+'/t.txt',t_file)
            np.savetxt(patht+'/fusiont.txt',fusion_time)

            sim=sim+1